In [1]:
from pytickersymbols import PyTickerSymbols

stock_data = PyTickerSymbols()
djs = [stock['symbol'] for stock in list(stock_data.get_stocks_by_index('DOW JONES'))]

In [2]:
stock_data.get_all_indices()

['NASDAQ 100',
 'SDAX',
 'MOEX',
 'DAX',
 'TECDAX',
 'CDAX',
 'EURO STOXX 50',
 'OMX Helsinki 25',
 'S&P 500',
 'CAC 40',
 'IBEX 35',
 'AEX',
 'S&P 600',
 'DOW JONES',
 'FTSE 100',
 'OMX Stockholm 30',
 'S&P 100',
 'Switzerland 20',
 'MDAX',
 'BEL 20',
 'CAC Mid 60']

Add technical indicators

Read bars dataset from Alpaca API

In [4]:
import pandas as pd
from alpaca_trade_api.rest import REST
import os
import numpy as np
import exchange_calendars as ec

# Retrieve the historical bar data for AAPL during regular trading hours. exclude aftermarket data.

# Replace with your own API credentials
# API_KEY = 'PKYLYX8BR5Z6F0TIMX92'
# API_SECRET = 'ZWHycpjUN02eJw4bcx8dZvOAHnwY2gsUcUtEpTRX'
# BASE_URL = 'https://api.alpaca.markets'
PAPER_URL = 'https://paper-api.alpaca.markets'

API_KEY = os.environ['ALPACA_API_KEY']
API_SECRET = os.environ['ALPACA_API_SECRET']
BASE_URL = os.environ['ALPACA_BASE_URL']
# PAPER_URL = os.environ['ALPACA_PAPER_URL']

# Create an instance of the REST API
api = REST(API_KEY, API_SECRET, PAPER_URL, api_version='v2')

interval = '1Min' # "1Min", "5Min", "15Min", "1H", "1D", "1W"
# interval = timedelta(minutes=1)
start = "2017-01-02"  # start time, min="2015-01-01", but exchange calendar limit is min=2017-01-01
end = "2017-03-01"  # End time, max=today's date
# symbol = ['AAPL', 'MMM', 'AXP'] # symbols should trade in ET 
symbol = ['DOW']

# Check if symbol trades in ET
pass

# df = api.get_bars(djs, interval, start, end, adjustment='raw', limit=10000).df
# df = df.tz_convert('America/New_York')

In [5]:
def preprocess(df, date, interval):
    # preprocesses data to insert any missing intervals during market hours and forward fills them
    start = pd.Timestamp(date, tz='America/New_York') + pd.Timedelta('9:30:00')
    end = pd.Timestamp(date, tz='America/New_York') + pd.Timedelta('15:59:00')
    index = pd.date_range(start=start, end=end, freq=interval)
    # creates rows for missing intervals
    resampled = df.reindex(index, method='ffill')
    if resampled.isna().all().all():
        raise ValueError('Data does not have entries in NYSE market hours.')
    # backward fill if first row is nan
    if resampled.isna().any().any():
        resampled = resampled.bfill()
    
    return resampled

In [6]:
def download(start, end, interval, dir = None):
    
    if dir is not None:
        if not os.path.exists(dir):
            os.makedirs(dir)
        else:
            if os.path.exists(os.path.join(dir, 'data.csv')):
                header = False
            else:
                header = True

    nyse = ec.get_calendar("NYSE")
    working_days = nyse.sessions_in_range(start, end).strftime('%Y-%m-%d')
    if dir is None:
        data = list()

    for day in working_days:
        # start and end are in UTC. ET is -04:00 from March to November and -05:00 otherwise.
        # We pad start by one hour to account for daylight saving time.
        # after tz conversion from Nov. to March, 8:30-9:30 is extra and from March to Nov. 16:00-17:00 is extra.
        # padded 1 hour is automatically dropped in resampling
        start = f'{day}T8:30:00-05:00'
        end = f'{day}T16:00:00-05:00'
        bars = api.get_bars(djs, interval, start, end, adjustment='raw', limit=None).df
        bars = bars.tz_convert('America/New_York')
        features = pd.concat([preprocess(group[1], day, interval) for group in bars.groupby('symbol')], axis = 1)
        features = features.select_dtypes(include=np.number)
        if dir is not None:
            features.to_csv(os.path.join(dir, 'data.csv'), index=True, mode='a', header = header)
            header = False
        else:
            data.append(features)
    
    return pd.concat(data) if dir is None else None
                

In [550]:
df

,open,high,low,close,volume,trade_count,vwap,open,high,low,...,volume,trade_count,vwap,open,high,low,close,volume,trade_count,vwap
2017-02-01 09:30:00-05:00,127.3800,128.290,126.880,128.1200,28836346,129536,127.541057,158.160,159.1100,157.0500,...,320852,1322,81.294764,66.7800,66.7800,66.040,66.540,1262507,6289,66.257831
2017-02-01 10:30:00-05:00,128.1300,128.450,127.660,128.1700,18052176,86666,128.066703,158.740,159.2200,157.7900,...,608214,4587,80.800793,66.5400,66.6100,66.240,66.310,1156496,8137,66.367341
2017-02-01 11:30:00-05:00,128.1681,128.480,127.850,128.3300,9915371,58226,128.186997,158.690,159.4500,158.5401,...,341864,3174,80.510667,66.3150,66.5700,66.300,66.506,815189,5669,66.460489
2017-02-01 12:30:00-05:00,128.3300,129.500,128.310,129.2150,10898926,56617,128.907904,159.050,159.5700,158.8800,...,250966,2531,80.453362,66.5100,66.6350,66.470,66.520,486591,3742,66.529831
2017-02-01 13:30:00-05:00,129.2150,129.430,128.940,129.0399,6949364,40010,129.236412,159.080,159.1900,158.3400,...,519449,4120,80.373175,66.5154,66.7139,66.480,66.560,1127760,6129,66.636185
2017-02-01 14:30:00-05:00,129.0300,130.120,129.000,129.9900,13981562,67884,129.699711,158.610,159.0000,158.4100,...,1204192,5377,80.498507,66.5500,66.7000,66.270,66.350,990042,6508,66.459572
2017-02-01 15:30:00-05:00,129.9900,130.490,128.750,128.7500,15982894,86045,129.731023,158.840,159.7500,158.7600,...,1319650,11800,81.072584,66.3500,66.3950,66.190,66.230,1783781,12146,66.284792
2017-02-02 09:30:00-05:00,128.3400,129.390,127.900,128.7800,7726894,37610,128.698728,159.790,160.0000,156.9100,...,290938,2126,80.769534,66.3000,67.1400,66.300,66.690,1186502,6040,66.808095
2017-02-02 10:30:00-05:00,128.7800,128.820,128.260,128.6887,5599969,36108,128.569933,159.000,159.3400,157.7500,...,517724,5856,80.416068,66.6850,66.7250,66.430,66.690,1259676,8809,66.616150
2017-02-02 11:30:00-05:00,128.6850,128.730,127.940,128.0050,4537268,25770,128.313094,158.930,159.3700,158.4200,...,292079,3106,80.699697,66.6850,67.0100,66.680,66.845,1110432,7299,66.889695


In [ ]:
trades = api.get_trades('AAPL', start, end).df

In [453]:
trades

,exchange,price,size,conditions,id,tape
timestamp,,,,,,
2017-05-01 08:00:00.052900+00:00,P,143.71,5,"[@, T, I]",2,C
2017-05-01 08:00:00.052900+00:00,P,143.71,46,"[@, T, I]",3,C
2017-05-01 08:08:13.925700+00:00,P,144.01,100,"[@, T]",17,C
2017-05-01 08:43:51.032600+00:00,T,144.01,4,"[@, F, T, I]",19,C
2017-05-01 10:03:45.951400+00:00,P,144.00,1,"[@, T, I]",22,C
...,...,...,...,...,...,...
2017-05-01 23:57:10.565700+00:00,T,146.85,50,"[@, F, T, I]",2337874,C
2017-05-01 23:57:11.649800+00:00,P,146.85,50,"[@, F, T, I]",2337876,C
2017-05-01 23:57:11.649800+00:00,T,146.80,300,"[@, F, T]",2337877,C


In [454]:
quotes = api.get_quotes('AAPL', start, end).df

sleep 3 seconds and retrying https://data.alpaca.markets/v2/stocks/AAPL/quotes 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v2/stocks/AAPL/quotes 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v2/stocks/AAPL/quotes 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v2/stocks/AAPL/quotes 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v2/stocks/AAPL/quotes 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v2/stocks/AAPL/quotes 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v2/stocks/AAPL/quotes 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v2/stocks/AAPL/quotes 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v2/stocks/AAPL/quotes 3 more time(s)...
sleep 3 seconds and retrying https://data.alpaca.markets/v2/stocks/AAPL/quotes 3 more time(s)...
sleep 3 seconds and retrying h

In [455]:
quotes

,ask_exchange,ask_price,ask_size,bid_exchange,bid_price,bid_size,conditions,tape
timestamp,,,,,,,,
2017-05-01 08:00:00.027000+00:00,T,0.00,0,T,1.00,1,[?],C
2017-05-01 08:00:00.187400+00:00,P,150.00,1,P,143.45,10,[?],C
2017-05-01 08:00:00.187400+00:00,T,0.00,0,P,143.45,10,[?],C
2017-05-01 08:00:00.187400+00:00,T,0.00,0,P,102.70,60,[?],C
2017-05-01 08:00:00.187400+00:00,P,148.00,1,P,143.45,10,[?],C
...,...,...,...,...,...,...,...,...
2017-05-01 23:57:53.257400+00:00,K,146.80,1,K,146.75,5,[?],C
2017-05-01 23:58:35.689100+00:00,K,146.80,1,T,146.76,1,[?],C
2017-05-01 23:59:10.780800+00:00,T,146.85,78,T,146.76,1,[?],C
